In [1]:
# !/usr/bin/env python3

#cd ~/Desktop/snet/gits/lang-learn-repo/alex_tests

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab
import sys  
import subprocess
import pywt as pywt

from sklearn import metrics
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA

In [2]:
path = '~/Desktop/snet/gits/lang-learn-repo/alex_tests/data'
filenames = ['bgpclear_first','bgpclear_second','bgpclear_apptraffic_2hourRun','portflap_first']
dims = 3
blocks = 3
syms = "ABCDEFGHIJKLMNOPQRSTUVWXYZ@"

In [3]:
def pcaSymbolize(df,dim=3,block=3):
    #divs = blocks**dim
    out = df.copy()
    
    for v in out.columns:
        out[v] = (out[v]-min(out[v]))/(max(out[v])-min(out[v]))
        
    out['sym']="_"
    
    for d in range(dim):
        out['sym'+str(d)]=0
        for b in range(block):
            out.loc[out.loc[:,out.columns[d]]>(b+1)/float(block),'sym'+str(d)]=b+1
    
    for i in df.index:
        out.loc[i,'sym']=syms[out.loc[i,'sym0']*9+out.loc[i,'sym1']*3+out.loc[i,'sym2']]
    return out
            

In [4]:
#for filename in filenames:
filename = filenames[0]
doc = pd.read_csv(path+"/"+filename+"/all_"+filename+"_raw.csv") 
clst = pd.read_csv(path+"/"+filename+"/"+filename+"_clusters.csv")

dataT = doc.copy()

for v in dataT.columns:
    dataT[v] = (dataT[v]-min(dataT[v]))/(max(dataT[v])-min(dataT[v]))

dataT = dataT.replace(np.nan,0)
dataT = dataT.T
data = dataT.iloc[:-1,:]
data['clst']=list(clst['cluster'])

for c in range(max(data['clst'])):
    sub = data.loc[data.loc[:,'clst']==c,data.columns[:-1]]
    sub = sub.T
    pca = PCA(n_components=dims)
    pca.fit(sub)
    newSub = pca.transform(sub)
    newSub = pd.DataFrame(newSub)
    newSub = pcaSymbolize(newSub,dims,blocks)

    data = data.T
    data['sym'+str(c)]=0
    data.loc[:len(newSub['sym']),'sym'+str(c)]=list(newSub['sym'])
    data = data.T
    #display(newSub.groupby('sym')['sym0'].count())
    #display(newSub)
    

/home/alexcasar/miniconda3/envs/ull/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
display(data)

,0,1,2,3,4,5,6,7,8,9,...,653,654,655,656,657,658,659,660,661,clst
HundredGigE0/0/0/0input-data-rate_leaf1,0.565805,0.564234,0.568302,0.569443,0.569832,0.568532,0.571207,0.57155,0.573751,0.571695,...,0.575165,0.57522,0.576865,0.577637,0.575872,0.573469,0.574441,0.57541,0.577192,7
HundredGigE0/0/0/0input-load_leaf1,0.566667,0.566667,0.566667,0.572222,0.572222,0.566667,0.572222,0.572222,0.572222,0.572222,...,0.577778,0.577778,0.577778,0.577778,0.577778,0.572222,0.572222,0.577778,0.577778,6
HundredGigE0/0/0/0input-packet-rate_leaf1,0.569099,0.568198,0.571566,0.572576,0.572992,0.571748,0.574147,0.574568,0.576569,0.574673,...,0.577959,0.578005,0.579611,0.580323,0.578779,0.576135,0.57717,0.578204,0.580053,7
HundredGigE0/0/0/0output-data-rate_leaf1,0.653834,0.65274,0.654111,0.653104,0.655767,0.656545,0.6579,0.656103,0.653819,0.659201,...,0.664021,0.66406,0.663287,0.662772,0.661262,0.663443,0.665862,0.667922,0.670653,7
HundredGigE0/0/0/0output-load_leaf1,0.65285,0.65285,0.65285,0.65285,0.658031,0.658031,0.658031,0.658031,0.65285,0.658031,...,0.663212,0.663212,0.663212,0.663212,0.663212,0.663212,0.668394,0.668394,0.673575,1
HundredGigE0/0/0/0output-packet-rate_leaf1,0.661196,0.660082,0.661607,0.660714,0.663203,0.663861,0.66515,0.663633,0.661523,0.666438,...,0.67025,0.670168,0.669357,0.668836,0.667466,0.669362,0.671782,0.673645,0.676218,7
HundredGigE0/0/0/1input-data-rate_leaf1,0.789165,0.790655,0.792331,0.789849,0.791928,0.792403,0.790342,0.788942,0.788534,0.790993,...,0.795262,0.795495,0.795935,0.79524,0.794766,0.789796,0.791877,0.792881,0.793348,3
HundredGigE0/0/0/1input-load_leaf1,0.78882,0.78882,0.795031,0.78882,0.795031,0.795031,0.78882,0.78882,0.78882,0.78882,...,0.795031,0.795031,0.795031,0.795031,0.795031,0.78882,0.795031,0.795031,0.795031,3
HundredGigE0/0/0/1input-packet-rate_leaf1,0.782215,0.783305,0.784887,0.782645,0.784511,0.78513,0.783327,0.781881,0.78153,0.783616,...,0.786878,0.787149,0.787483,0.787004,0.786694,0.781973,0.783966,0.784852,0.785307,3
HundredGigE0/0/0/1output-data-rate_leaf1,0.704538,0.703017,0.700321,0.702848,0.70457,0.705167,0.706433,0.704638,0.705084,0.703729,...,0.671212,0.67142,0.670826,0.672312,0.675271,0.673094,0.675957,0.675806,0.674486,7


In [8]:
output = data.T
output = output.loc[output.index[:-1],output.columns[-7:]]
display(output)

,sym0,sym1,sym2,sym3,sym4,sym5,sym6
0,B,D,B,H,B,E,B
1,B,D,B,H,B,E,B
2,B,D,B,H,E,E,B
3,B,D,B,H,B,E,B
4,B,D,B,H,E,B,B
5,B,D,B,H,B,E,B
6,B,D,B,H,B,B,B
7,B,D,B,H,B,E,B
8,B,D,B,H,B,E,B
9,B,D,B,H,B,E,B


In [11]:
output['anomaly']=list(doc['anomaly'])
outSY = output[output['anomaly']==1]
outSN = output[output['anomaly']==0]

In [28]:
n=3

print('anomaly')
print(outSY.groupby(by=outSY.columns[n]).count().iloc[:,0])
print('non anomaly')
print(outSN.groupby(by=outSN.columns[n]).count().iloc[:,0])

anomaly
sym3
A      9
B    105
C     17
D     19
E     83
H     18
J      7
K      4
L      5
M      7
N      2
Q      6
U      4
V     14
X      7
Name: sym0, dtype: int64
non anomaly
sym3
A      1
B    247
E     32
H     75
Name: sym0, dtype: int64
